Get YOLO Outputs for Each Image

In [5]:
from ultralytics import YOLO
import numpy as np
import os

# Load model
model = YOLO('insect_dataset_split/runs/detect/yolov8s_insect_detection/weights/best.pt')

# List of image paths (should match your 250 symptom rows)
image_folder = 'insect_dataset_split/images/train'  # or your chosen folder
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])[:640]

yolo_scores = []
for img_path in image_files:
    results = model(img_path)
    # Get max confidence for any detection in the image, or 0 if no detection
    if len(results[0].boxes) > 0:
        conf = float(results[0].boxes.conf.max())
    else:
        conf = 0.0
    yolo_scores.append(conf)

yolo_scores = np.array(yolo_scores).reshape(-1, 1)
np.save('yolo_insect_confidences.npy', yolo_scores)


image 1/1 c:\Users\hemes\Desktop\AGRITHON\insect_dataset_split\images\train\insect1.jpg: 640x640 1 insect, 132.6ms
Speed: 5.7ms preprocess, 132.6ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\hemes\Desktop\AGRITHON\insect_dataset_split\images\train\insect10.jpg: 640x640 2 insects, 14.2ms
Speed: 4.9ms preprocess, 14.2ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\hemes\Desktop\AGRITHON\insect_dataset_split\images\train\insect10_aug_0.jpg: 640x640 2 insects, 13.3ms
Speed: 3.8ms preprocess, 13.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\hemes\Desktop\AGRITHON\insect_dataset_split\images\train\insect10_aug_1.jpg: 640x640 2 insects, 15.5ms
Speed: 4.4ms preprocess, 15.5ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\hemes\Desktop\AGRITHON\insect_dataset_split\images\train\insect10_aug_10.jpg: 640x640 2 insects, 14.4ms
Speed: 3

 Get TabNet Outputs for Each Symptom Row

In [ ]:
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier

# Load symptom data (ensure it matches the images)
symptom_df = pd.read_csv('insect_symptom_dataset_640.csv').iloc[:640]
X_symptoms = symptom_df.drop(columns=['Insect_Present_Label']).values
y_labels = symptom_df['Insect_Present_Label'].values

# Load TabNet model
tabnet_model = TabNetClassifier() 
tabnet_model.load_model('tabnet_insect.zip.zip')

tabnet_probs = tabnet_model.predict_proba(X_symptoms)[:, 1].reshape(-1, 1)
np.save('tabnet_insect_probs.npy', tabnet_probs)
np.save('insect_labels.npy', y_labels)

c:\Users\hemes\anaconda3\envs\agri\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
c:\Users\hemes\anaconda3\envs\agri\lib\site-packages\pytorch_tabnet\abstract_model.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

Build the Fusion Dataset

In [27]:

yolo_scores = np.load('yolo_insect_confidences.npy')
tabnet_probs = np.load('tabnet_insect_probs.npy')
y_labels = np.load('insect_labels.npy')
yolo_weight = 2.0  # Increase this value to give more priority to YOLO
fusion_features = np.concatenate([yolo_scores * yolo_weight, tabnet_probs], axis=1)

Train and Evaluate a Fusion Classifier

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

X_train, X_val, y_train, y_val = train_test_split(fusion_features, y_labels, test_size=0.2, random_state=42)

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
clf = MLPClassifier(hidden_layer_sizes=(16, 8), max_iter=200, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print("Fusion Model Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))

Fusion Model Accuracy: 0.8125
Precision: 0.8125
Recall: 1.0
F1 Score: 0.896551724137931


Save and Load Fusion Model

In [29]:
import joblib
joblib.dump(clf, 'fusion_classifier.joblib')
# To load: clf = joblib.load('fusion_classifier.joblib')

['fusion_classifier.joblib']

## Inference: Predict for a New Image and Symptom Input
This cell demonstrates how to upload an image and input symptoms, then get YOLO, TabNet, and fusion predictions.

In [30]:
from ultralytics import YOLO
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
import pandas as pd
import joblib

# Load models (do this only once)
yolo_model = YOLO('insect_dataset_split/runs/detect/yolov8s_insect_detection/weights/best.pt')
tabnet_model = TabNetClassifier()
tabnet_model.load_model('tabnet_insect.zip.zip')
fusion_clf = joblib.load('fusion_classifier.joblib')  # Save your trained fusion model as shown earlier
def predict_insect(image_path, symptom_answers, yolo_weight=2.0):
    """
    image_path: str, path to the image file
    symptom_answers: list or np.array of 30 binary values (0/1)
    """
    # 1. YOLO inference
    yolo_result = yolo_model(image_path)
    if len(yolo_result[0].boxes) > 0:
        yolo_conf = float(yolo_result[0].boxes.conf.max())
    else:
        yolo_conf = 0.0

    # 2. TabNet inference
    X_symptom = np.array(symptom_answers).reshape(1, -1)
    tabnet_prob = tabnet_model.predict_proba(X_symptom)[0, 1]

    # 3. Fusion (apply YOLO weight)
    fusion_input = np.array([[yolo_conf * yolo_weight, tabnet_prob]])
    fusion_pred = fusion_clf.predict(fusion_input)[0]
    fusion_proba = fusion_clf.predict_proba(fusion_input)[0, 1]

    print(f"YOLO confidence (weighted): {yolo_conf * yolo_weight:.2f}")
    print(f"YOLO confidence (unweighted): {yolo_conf:.2f}")
    print(f"TabNet probability: {tabnet_prob:.2f}")
    print(f"Fusion prediction: {fusion_pred} (probability: {fusion_proba:.2f})")
    return fusion_pred, fusion_proba
# Example usage:
# image_path = 'insect_dataset_split/images/train/your_image.jpg'
# symptom_answers = [0, 1, 0, ..., 1]  # 30 binary values
# predict_insect(image_path, symptom_answers)

c:\Users\hemes\anaconda3\envs\agri\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
c:\Users\hemes\anaconda3\envs\agri\lib\site-packages\pytorch_tabnet\abstract_model.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

**Instructions:**  
- Replace `image_path` with the path to your test image.
- Replace `symptom_answers` with a list of 30 binary values for the symptoms.
- Run the function to get YOLO, TabNet, and fusion predictions for your sample.

In [32]:
test_symptoms = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
test_image_path = "test_insect/14.png"
predict_insect(test_image_path, test_symptoms)


image 1/1 c:\Users\hemes\Desktop\AGRITHON\test_insect\14.png: 384x640 (no detections), 84.4ms
Speed: 3.6ms preprocess, 84.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
YOLO confidence (weighted): 0.00
YOLO confidence (unweighted): 0.00
TabNet probability: 0.63
Fusion prediction: 1 (probability: 0.72)


(np.int64(1), np.float64(0.7234991466279834))